In [38]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

In [39]:
deva_images_list = tf.data.Dataset.list_files('data/Devasheesh/*.jpg', shuffle=True)
swar_images_list = tf.data.Dataset.list_files('data/Swarnim/*.jpg', shuffle=True)
neg_images_list = tf.data.Dataset.list_files('data/Unknown/*/*.jpg', shuffle=True)

# combine all the images into one dataset
images_list = deva_images_list.concatenate(swar_images_list).concatenate(neg_images_list)

# shuffle the dataset
images_list = images_list.shuffle(buffer_size=1000)

In [76]:
#! see values inside variables
# for i in images_list.take(5):
#     print(i.numpy())

# print(len(images_list))
print(type(images_list))

b'data\\Unknown\\n000040\\0163_02.jpg'
b'data\\Swarnim\\Swarnim_72.jpg'
b'data\\Unknown\\n000001\\0179_01.jpg'
b'data\\Unknown\\n000040\\0151_02.jpg'
b'data\\Unknown\\n000040\\0323_04.jpg'
<class 'tensorflow.python.data.ops.dataset_ops.ShuffleDataset'>


In [59]:
# take the image path and return the label
def get_label(image_path):
    parts = tf.strings.split(image_path, os.path.sep)
    if parts[1] == 'Devasheesh':
        return [0, 1, 0] 
        # return 'devasheesh'
    elif parts[1] == 'Swarnim':
        return [0, 0, 1]
        # return 'swarnim'
    elif parts[1] == 'Unknown':
        return [1, 0, 0]
        # return 'unknown'
    else:
        return [0, 0, 0]
        # return 'unknown'


In [ ]:
get_label('data\\Unknown\\n000029\\0145_01.jpg')


In [74]:
# takes the image path and returns the the face as a tensor
def preprocess_image(image_path):
    label = get_label(image_path)
    # label = tf.cast(label, tf.float32)
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=1)
    image = tf.image.resize(image, [360, 480])
    # ? extract the face from the image using dataframe
    image = tf.cast(image, tf.float32)
    # label = tf.reshape(label, image.shape)
    image = image /255.0
    return image, tf.constant(label)
    # return image

In [75]:
# image, label = preprocess_image('data\\Unknown\\n000029\\0145_01.jpg')
print(type(preprocess_image('data\\Unknown\\n000029\\0145_01.jpg')))
# plt.imshow(image, cmap='gray')
# plt.title(label)
# image

<class 'tuple'>


In [72]:
# dataset object

# Define a generator function that yields each file path
def generator():
    for filepath in images_list:
        yield filepath.numpy()


dataset = tf.data.Dataset.from_generator(generator, output_types=tf.string)
print(dataset)
dataset2 = dataset.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)


<FlatMapDataset element_spec=TensorSpec(shape=<unknown>, dtype=tf.string, name=None)>


TypeError: in user code:

    File "C:\Users\swast\AppData\Local\Temp\ipykernel_19448\910426502.py", line 13, in preprocess_image  *
        return tf.convert_to_tensor((image, tf.constant(label)))

    TypeError: Expected any non-tensor type, but got a tensor instead.


In [ ]:
for img, label in dataset.take(3):
    print(img.shape)
    plt.imshow(img, cmap='gray')
    plt.title(label)
    plt.show()